<a href="https://colab.research.google.com/github/vmadalasa/EVAS4/blob/master/Copy_of_Madhu_session499_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
#second set

class Net(nn.Module):
    def __init__(self):
        print('set 2')
        #super(Net, self).__init__()
        super(Net, self).__init__()

       
        #extracted receptive fld 3X3
        #we need to get to 5X5 for MNIST
        #If we start at 32 channels for 1st step, the second one at 64 channels exceeds 20,000 parameters. So reducing conv 1 to 16 c

        
        self.batch3=nn.BatchNorm2d(32)
        self.batch2=nn.BatchNorm2d(16)    
        self.batch1=nn.BatchNorm2d(8)  
        self.drop1=nn.Dropout(0.1)   
        self.drop2=nn.Dropout(0.1)   
        self.drop3=nn.Dropout(0.1)          

       #First conv set Edges and gradients
       #-----------------------------------#
        self.conv1 = nn.Conv2d(1, 16, 3)       #input = 28X28X1 OUtput: 26X26X16 RF 3X3 parameters = (kernel*input channels +1)*opchannels
                                                #parameters = (3*3*1+1)*16 = 160
        self.conv3 = nn.Conv2d(16, 32, 3)      #input 26X26X16 OUtput: 24X24X32 RF 5X5 params - (3*3*16 +1)*32 = 145*32

        #First pool layer
        self.pool1 = nn.MaxPool2d(2, 2)                   #input 24X24X32 o/p 12X12X32 RF 10X10 #conv3-conv1 pool . zero (why)

        
        #second set of layers - Textures and patterns layer#
        #-----------------------------#

        #1X1 reduction layer

        self.conv4 = nn.Conv2d(32, 8, 1)             #input 12X12X32 OUtput: 12X12X8 RF 10X10  parameters - (1X1*32+1)*8 = 33*8=264

        #1st conv
        self.conv5 = nn.Conv2d(8, 16, 3, padding=1)             #input 12X12X8 OUtput: 12X12X16 RF 12X12 params = (3*3*8+1) =1168
  
      

        #2nd conv layer
        self.conv7 = nn.Conv2d(16, 32, 3)             #input 12X12X16 OUtput: 10X10X32 RF 14X14  params : (3*3*16 +1)*32 = 145*32=4640

        #2nd pool
        self.pool2 = nn.MaxPool2d(2, 2)               #input 10X10X32 OUtput: 5X5X32 RF 28X28 #conv3-conv4 pool 

        
        #Third set of layers - Poo and objects together
        #----------------------------------_#
       #self.conv7 = nn.Conv2d(32, 8, 1)               #input 7X7X32 OUtput: 7X7X8 RF 28X28 . 

        self.conv8 = nn.Conv2d(32, 16, 1)             #input 5X5X32 OUtput: 5X5X16 RF 28X28 pramaeters (32+1)*16 = 528

        self.conv9 = nn.Conv2d(16, 32, 3)              #input 5X5X16 OUtput: 3X3X32 RF 30X30  (3*3*16 +1)*32 = 145*32=4640

       
        
       

        #self.conv10 = nn.Conv2d(32, 16, 1)               #input 3X3X32 OUtput: 3X3X16 RF 32X32


        
        self.conv11 = nn.Conv2d(32, 10, 3)          #now reduce the 32 channels to 10 channels using a 3X3 conv.
        self.gap = nn.AdaptiveAvgPool2d((1,1))
        #self.gap = nn.AvgPool2d(2,2)


    def forward(self, x):
        
        
        
        x = self.pool1(self.drop1(F.relu(self.conv3(self.drop2(F.relu(self.conv1(x)))))))
        
        x = self.pool2(self.drop1(F.relu(self.conv7(self.drop1(F.relu(self.conv5(self.drop2(F.relu(self.conv4(x))))))))))
                                        
        
        #x = self.pool2(F.relu(self.conv7()
        #x = self.drop(F.relu(self.conv6(self.drop(F.relu(self.conv5(x))))))
        #x = self.drop(F.relu(self.conv11(self.drop(F.relu(self.conv9(self.drop(F.relu(self.conv8(self.drop(F.relu(self.conv7(x)))))))))))) #)))
        #x = self.conv7(x)
        x = self.drop1(F.relu(self.conv9(self.drop1(F.relu(self.conv8(x))))))   
        #new
        x = (self.conv11(x))
        
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [5]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

set 2
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
           Dropout-2           [-1, 16, 26, 26]               0
            Conv2d-3           [-1, 32, 24, 24]           4,640
           Dropout-4           [-1, 32, 24, 24]               0
         MaxPool2d-5           [-1, 32, 12, 12]               0
            Conv2d-6            [-1, 8, 12, 12]             264
           Dropout-7            [-1, 8, 12, 12]               0
            Conv2d-8           [-1, 16, 12, 12]           1,168
           Dropout-9           [-1, 16, 12, 12]               0
           Conv2d-10           [-1, 32, 10, 10]           4,640
          Dropout-11           [-1, 32, 10, 10]               0
        MaxPool2d-12             [-1, 32, 5, 5]               0
           Conv2d-13             [-1, 16, 5, 5]             528
          Dropout-14             

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:90: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


param'


In [6]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...



Done!


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:

model = Net().to(device)
#optimizer = optim.SGD(model.parameters(), lr=0.009, momentum=0.91)
optimizer = optim.Adam(model.parameters(), lr=0.001,betas=(0.9,0.99))

#for epoch in range(1, 2):
for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

set 2


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:90: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0773, Accuracy: 9779/10000 (97.79%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0180, Accuracy: 9943/10000 (99.43%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0185, Accuracy: 9932/10000 (99.32%)



loss=0.009070225059986115 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 42.51it/s]



Test set: Average loss: 0.0175, Accuracy: 9941/10000 (99.41%)



In [0]:
Results Matrix
#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)  # Accuracy 38% #initial
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.6) #Accuracy 32%
#optimizer = optim.Adam(model.parameters(), lr=0.001,betas=(0.9,0.99)) #Accuracy 31%
#optimizer = optim.Adam(model.parameters(), lr=0.0001,betas=(0.9,0.99)) #Accuracy 49%
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.99) ; for epoch in range(1, 3): 89%
#dropout after each laye, batch norm after each layer - 95.4% - 97%
#removed batch norm from last layers
#relu channels to be redone after last change ..

#Final Arch - edges and patterns conv set- textures and gradients conv set;  



In [0]:
#first iter set  - pl do not use
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 8, 3) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(8, 16, 3)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv_11 = nn.Conv2d(16, 8, 1)
        self.conv3 = nn.Conv2d(8, 16, 3)
        self.conv4 = nn.Conv2d(16, 32, 3)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv_12 = nn.Conv2d(32, 8, 1)
        self.conv5 = nn.Conv2d(8, 16, 3)
        self.conv6 = nn.Conv2d(16, 32, 3)
        self.conv7 = nn.Conv2d(32, 10, 3)
        
        self.drop = nn.Dropout2d(p=0.1)
        self.batch_8 = nn.BatchNorm2d(8)
        self.batch_16 = nn.BatchNorm2d(16)
        self.batch_32 = nn.BatchNorm2d(32)
        #self.batch_64 = nn.BatchNorm2d(64)
        self.gap = nn.AvgPool2d(2,2)
      

    def forward(self, x):
        x = self.drop(self.conv_11(self.pool1(self.batch_16(F.relu(self.conv2(self.drop(self.batch_8(F.relu(self.conv1(x))))))))))
        x = self.drop(self.conv_12(self.batch_32(F.relu(self.conv4(self.drop(self.batch_16(F.relu(self.conv3(x)))))))))
        x = self.drop(F.relu(self.conv6(self.drop(self.batch_16(F.relu(self.conv5(x)))))))
        x = self.gap(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)